In [1]:
# imports
from google.cloud import vision
from PIL import Image, ImageDraw, ImageFont, ImageFilter

In [6]:
def detect_text(path):
    """Detects text in the file."""
    from google.cloud import vision

    client = vision.ImageAnnotatorClient()

    with open(path, "rb") as image_file:
        content = image_file.read()

    image = vision.Image(content=content)
    
    items = list()

    response = client.text_detection(image=image)
    texts = response.text_annotations

    for text in texts:
        v = list()
        for vertex in text.bounding_poly.vertices:
            v.append({"x": vertex.x, "y": vertex.y})
        items.append({"label": text.description, "vertices": v})

    if response.error.message:
        raise Exception(
            "{}\nFor more info on error messages, check: "
            "https://cloud.google.com/apis/design/errors".format(response.error.message)
        )
    return items

def is_box_contained(box1, box2):
    """
    Check if box1 is contained within box2.
    
    Args:
    - box1: List of vertices (dictionaries with 'x' and 'y' coordinates) for box1.
    - box2: List of vertices for box2.
    
    Returns:
    - True if box1 is fully contained within box2, False otherwise.
    """
    x1_min = min(v['x'] for v in box1)
    y1_min = min(v['y'] for v in box1)
    x1_max = max(v['x'] for v in box1)
    y1_max = max(v['y'] for v in box1)

    x2_min = min(v['x'] for v in box2)
    y2_min = min(v['y'] for v in box2)
    x2_max = max(v['x'] for v in box2)
    y2_max = max(v['y'] for v in box2)

    # Check if all corners of box1 are within the bounds of box2
    return (x1_min >= x2_min and x1_max <= x2_max and y1_min >= y2_min and y1_max <= y2_max)

def draw_bounding_boxes(image_path, bounding_boxes, output_path):
    """
    Draws bounding boxes with labels on an image, skipping larger boxes that contain smaller ones.
    
    Args:
    - image_path: Path to the image file.
    - bounding_boxes: List of dictionaries containing 'label' and 'vertices'.
                      The 'vertices' should be in pixel coordinates.
    - output_path: Path to save the modified image.
    """
    # Open the image file
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    
    # Optional: Load a font (you can use a default font if PIL doesn't have a custom font)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except IOError:
        font = ImageFont.load_default()
    
    # List to store boxes that are not large containers
    boxes_to_draw = []

    # Loop through each box
    for i, box in enumerate(bounding_boxes):
        is_container = False
        vertices = box['vertices']
        
        # Check if the current box contains any other smaller box
        for j, other_box in enumerate(bounding_boxes):
            if i != j and is_box_contained(other_box['vertices'], vertices):
                # If another box is contained within this one, mark this box as a container
                is_container = True
                break
        
        # If the box is not a container for other boxes, we keep it for drawing
        if not is_container:
            boxes_to_draw.append(box)
    
    # Now draw only the non-container boxes
    for box in boxes_to_draw:
        label = box['label']
        vertices = box['vertices']
        
        # Draw the box
        points = [(v['x'], v['y']) for v in vertices]
        draw.polygon(points, outline="red", width=3)
        
        # Get the top-left point for the label
        top_left = points[0]
        
        # Draw the label in the top-left corner of the bounding box
        draw.text(top_left, label, fill="red", font=font)
    
    # Save the modified image
    image.save(output_path)
    print(f"Image saved to: {output_path}")

def blur_bounding_boxes(image_path, bounding_boxes, output_path):
    """
    Blurs regions inside bounding boxes on an image, skipping larger boxes that contain smaller ones.
    
    Args:
    - image_path: Path to the image file.
    - bounding_boxes: List of dictionaries containing 'label' and 'vertices'.
                      The 'vertices' should be in pixel coordinates.
    - output_path: Path to save the modified image.
    """
    # Open the image file
    image = Image.open(image_path)
    
    # Filter out only the smaller boxes (not the larger containers)
    boxes_to_blur = []
    
    # Loop through each box and check if it is contained within another box
    for i, box in enumerate(bounding_boxes):
        is_container = False
        vertices = box['vertices']
        
        for j, other_box in enumerate(bounding_boxes):
            if i != j and is_box_contained(other_box['vertices'], vertices):
                # If another box is contained within this box, mark this box as a container
                is_container = True
                break
        
        # If the box is not a container, add it to the list for blurring
        if not is_container:
            boxes_to_blur.append(box)
    
    # Apply blur to the non-container boxes
    for box in boxes_to_blur:
        vertices = box['vertices']
        
        # Calculate the bounding box coordinates
        x_min = min(v['x'] for v in vertices)
        y_min = min(v['y'] for v in vertices)
        x_max = max(v['x'] for v in vertices)
        y_max = max(v['y'] for v in vertices)
        
        # Crop the region to blur
        region_to_blur = image.crop((x_min, y_min, x_max, y_max))
        
        # Apply the blur filter to the region
        blurred_region = region_to_blur.filter(ImageFilter.GaussianBlur(radius=10))
        
        # Paste the blurred region back into the image
        image.paste(blurred_region, (x_min, y_min, x_max, y_max))
    
    # Save the modified image
    image.save(output_path)
    print(f"Blurred image saved to: {output_path}")

In [7]:
#######################################################
# detect the text boxes
#######################################################
image_path = "doorstep_2.png"
bounding_boxes = detect_text(image_path)

In [8]:
#######################################################
# draws bounding boxes
#######################################################
# Path to save the modified image
output_path = "output_without_container_boxes.png"

# Draw bounding boxes with labels and save the image
draw_bounding_boxes(image_path, bounding_boxes, output_path)

Image saved to: output_without_container_boxes.png


In [9]:
#######################################################
# blurs
#######################################################
# Path to save the modified image
output_path = "blurred_output.png"

# Blur the regions inside the bounding boxes
blur_bounding_boxes(image_path, bounding_boxes, output_path)

Blurred image saved to: blurred_output.png
